In [1]:
from pybtex.database import parse_file
import json
from pprint import pprint

In [2]:
to_import = [['Romain','Vuillemot'],['Audrey','Serna'],['Christine','Michel'],['Karim','Sehaba'],['Elise','Lavoué']]
ref_list = []
for i in to_import:
    ref_list += [i[0] + ' ' + i[1]]

In [7]:
to_work_on = ['Elise','Lavoué']

In [3]:
# name filtering

def name_filtering(data,test):
    for j in test :
        dataset = []
        if 'author' in test[j].persons:
            str = 'author'
        elif 'editor' in test[j].persons: 
            str = 'editor'
        else: 
            continue
        for i in test[j].persons[str]:
            first = ""
            middle = ""
            last = ""
            line = ""
            all = ""
            if i.first_names:
                all += i.first_names[0]
            if i.middle_names:
                all += " " + i.middle_names[0]
            if i.last_names:
                all += " " + i.last_names[0]
            if i.lineage_names:
                all += " " + i.lineage_names[0]
            dataset.append(all)
        data[j] = dataset
    return data

In [4]:
def setup(data,ref_list):
    jfile = {}
    jfile["nodes"] = []
    jfile["links"] = []
    for article in data :
        to_do = False
        for k in data[article]:
            if k in ref_list:
                to_do = True
            if to_do:
                for i in data[article]:
                    node = {}
                    node["id"] = i
                    node["group"] = 1
                    if node not in jfile["nodes"]:
                        jfile["nodes"].append(node)
                    for j in data[article] :
                        if (i != j) :
                            link = {}
                            link["source"] = i
                            link["target"] = j
                            if link not in jfile["links"]:
                                jfile["links"].append(link)
    return jfile


In [5]:
def save(jfile):
    with open('test.json', 'w') as f:
        json.dump(jfile, f)

    pprint(jfile)

In [44]:
def clean_data(data):
    str_e1 = '{\\\'{e}}'
    str_e1_maj = '{\\\'{E}}'
    str_tiret = '{-}'
    str_o1 = '{\\^{o}}'
    str_e2 = '{\\"{e}}'
    str_u1 = '{\\"{u}}'
    str_e3 = '{\\`{e}}'
    str_o2 = '{\\`{o}}'
    str_a1 = '{\\"{a}}'
    str_ae = '{\\ae}'
    str_ss = '{\\ss}'
    str_o3 = '{\\"{o}}'
    str_i1 = '{\\"{\\i}}'
    str_i2 = '{\\\'{\\i}}'
    str_a2 = '{\\\'{a}}'
    str_i3 = '{\\^{\\i}}'
    str_n1 = '{\\~{n}}'
    
    
    for article in data :
        for i in range(len(data[article])):
            data[article][i] = data[article][i].replace(str_e1,'é')
            data[article][i] = data[article][i].replace(str_tiret,'-')
            data[article][i] = data[article][i].replace(str_o1,'ô')
            data[article][i] = data[article][i].replace(str_e2,'ë')
            data[article][i] = data[article][i].replace(str_u1,'ü')
            data[article][i] = data[article][i].replace(str_e1_maj,'É')
            data[article][i] = data[article][i].replace(str_e3,'è')
            data[article][i] = data[article][i].replace(str_o2,'ò')
            data[article][i] = data[article][i].replace(str_a1,'ä')
            data[article][i] = data[article][i].replace(str_ae,'æ')
            data[article][i] = data[article][i].replace(str_ss,'ß')
            data[article][i] = data[article][i].replace(str_o3,'ö')
            data[article][i] = data[article][i].replace(str_i1,'ï')
            data[article][i] = data[article][i].replace(str_i2,'í')
            data[article][i] = data[article][i].replace(str_a2,'à')
            data[article][i] = data[article][i].replace(str_i3,'î')
            data[article][i] = data[article][i].replace(str_n1,'ñ')

In [45]:
filtered_data = {}

for people in to_import:
    data = parse_file('bibtex_data/' + people[1] + '_' + people[0] + '.bib').entries
    filtered_data = name_filtering(filtered_data,data)
    clean_data(filtered_data)

jfile = setup(filtered_data,ref_list)
save(jfile)

{'links': [{'source': 'Christophe Hurter', 'target': 'Nathalie Henry Riche'},
           {'source': 'Christophe Hurter', 'target': 'Steven M. Drucker'},
           {'source': 'Christophe Hurter', 'target': 'Maxime Cordeil'},
           {'source': 'Christophe Hurter', 'target': 'Richard Alligier'},
           {'source': 'Christophe Hurter', 'target': 'Romain Vuillemot'},
           {'source': 'Nathalie Henry Riche', 'target': 'Christophe Hurter'},
           {'source': 'Nathalie Henry Riche', 'target': 'Steven M. Drucker'},
           {'source': 'Nathalie Henry Riche', 'target': 'Maxime Cordeil'},
           {'source': 'Nathalie Henry Riche', 'target': 'Richard Alligier'},
           {'source': 'Nathalie Henry Riche', 'target': 'Romain Vuillemot'},
           {'source': 'Steven M. Drucker', 'target': 'Christophe Hurter'},
           {'source': 'Steven M. Drucker', 'target': 'Nathalie Henry Riche'},
           {'source': 'Steven M. Drucker', 'target': 'Maxime Cordeil'},
           {'sour